<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 7:** Retrieval-Augmented Generation with Vector Stores</font>

<br>

In the previous notebook, we learned about embedding models and exercised some of their capabilities. We discussed their intended use cases of longer-form document comparison and found ways to use it as a backbone for more custom semantic comparisons. This notebook will progress these ideas toward the retrieval model's intended use case and explore how to build chatbot systems that rely on *vector stores* to automatically save and retrieve information.

<br>

### **Learning Objectives:**

- Understand how semantic-similarity-backed systems can facilitate easy-to-use retrieval formulations.

- Learn how to incorporate retrieval modules into your chat model systems for a retrieval-augmented generation (RAG) pipeline, which can be applied to tasks like document retrieval and conversation memory buffers.

<br>

### **Questions To Think About:**

- This notebook does not attempt to incorporate hierarchical reasoning or non-naive RAG (such as planning agents). Consider what modifications would be necessary to make these components work in an LCEL chain.

- Consider when it would be best to move your vector store solution into a scalable service and when a GPU will become necessary for optimization.

<br>

### **Environment Setup:**

In [4]:
# %%capture
## ^^ Comment out if you want to see the pip install process

## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [5]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

----

<br>

## Part 1: Summary of RAG Workflows

This notebook will explore several paradigms and derive reference code to help you approach some of the most common retrieval-augmented workflows. Specifically, the following sections will be covered (with the differences highlighted):

<br>

> ***Vector Store Workflow for Conversational Exchanges:***
- Generate semantic embedding for each new conversation.
- Add the message body to a vector store for retrieval.
- Query the vector store for relevant messages to fill in the LLM context.

<br>

> ***Modified Workflow for an Arbitrary Document:***
- **Divide the document into chunks and process them into useful messages.**
- Generate semantic embedding for each **new document chunk**.
- Add the **chunk bodies** to a vector store for retrieval.
- Query the vector store for relevant **chunks** to fill in the LLM context.
    - ***Optional:* Modify/synthesize results for better LLM results.**

<br>

> **Extended Workflow for a Directory of Arbitrary Documents:**
- Divide **each document** into chunks and process them into useful messages.
- Generate semantic embedding for each new document chunk.
- Add the chunk bodies to **a scalable vector database for fast retrieval**.
    - ***Optional*: Exploit hierarchical or metadata structures for larger systems.**
- Query the **vector database** for relevant chunks to fill in the LLM context.
    - *Optional:* Modify/synthesize results for better LLM results.

<br>

Some of the most important terminology surrounding RAG is covered in detail on the [**LlamaIndex Concepts page**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html), which itself is a great starting point for progressing towards the LlamaIndex loading and retrieving strategy. We highly recommend using it as a reference as you continue with this notebook and advise you to try out LlamaIndex after the course to consider the pros and cons firsthand!


<!-- > <img src="https://drive.google.com/uc?export=view&id=1cFbKbVvLLnFPs3yWCKIuzXkhBWh6nLQY" width=1200px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/data_connection_langchain.jpeg" width=1200px/>
>
> From [**Retrieval | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/)

----

<br>

## **Part 2:** RAG for Conversation History

In our previous explorations, we delved into the capabilities of document embedding models and used them to embed, store, and compare semantic vector representations of text. Though we could motivate how to efficiently extend this into vector store land manually, the true beauty of working with a standard API is its strong incorporation with other frameworks that can already do the heavy lifting for us!

<br>

### **Step 1**: Getting A Conversation

Consider a conversation crafted using Llama-13B between a chat agent and a blue bear named Beras. This dialogue, dense with details and potential diversions, provides a rich dataset for our study:


In [6]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

Using the manual embedding strategy from the previous notebook is still very viable, but we can also rest easy and let a **vector store** do all that work for us!

<br>

### **Step 2:** Constructing Our Vector Store Retriever

To streamline similarity queries on our conversation, we can employ a vector store to help keep track of passages for us! **Vector Stores**, or vector storage systems, abstract away most of the low-level details of the embedding/comparison strategies and provide a simple interface to load and compare vectors.


<!-- > <img src="https://drive.google.com/uc?export=view&id=1ZjwYbSZzsXK6ZP8O1-cY3BeRffV4oqzb" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/vector_stores.jpeg" width=1200px/>
>
> From [**Vector Stores | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/)

<br>

In addition to simplifying the process from an API perspective, vector stores also implement connectors, integrations, and optimizations under the hood. In our case, we will start with the [**FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss), which integrates a LangChain-compatable Embedding model with the [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss) library to make the process fast and scalable on our local machine!

**Specifically:**

1. We can feed our conversation into [**a FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss) via the `from_texts` constructor. This will take our conversational data and the embedding model to create a searchable index over our discussion.
2. This vector store can then be "interpreted" as a retriever, supporting the LangChain runnable API and returning documents retrieved via an input query.

The following shows how you can construct a FAISS vector store and reinterpret it as a retriever using the LangChain `vectorstore` API:

In [7]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 24.9 ms, sys: 0 ns, total: 24.9 ms
Wall time: 1.12 s


The retriever can now be used like any other LangChain runnable to query the vector store for some relevant documents:

In [8]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        id='7094f919-9ab7-48ac-ac99-f22c41f88e69',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='b61ed345-2731-449c-bb4a-4ccdbe5548e9',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        id='cafcec6f-0add-45ea-8c15-dac412cd10db',
        metadata={},
        page_content='[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for 
you![Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.'
    ),
    Document(
        id='10e1543c-701c-4a63-a2cd-21b18a5dc645',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

In [9]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        id='f3af6896-4c28-48ce-8f8a-b547e06ff458',
        metadata={},
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        id='10e1543c-701c-4a63-a2cd-21b18a5dc645',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        id='7094f919-9ab7-48ac-ac99-f22c41f88e69',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='b61ed345-2731-449c-bb4a-4ccdbe5548e9',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

As we can see, our retriever found a handful of semantically relevant documents from our query. You may notice that not all of the documents are useful or clear on their own. For example, a retrieval of *"Beras"* for *"your name"* may be problematic for the chatbot if provided out of context. Anticipating the potential problems and creating synergies between your LLM components can increase the likelihood of good RAG behavior, so keep an eye out for such pitfalls and opportunities.

<br>

### **Step 3:** Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [10]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [11]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

Based on the context provided, Beras lives in the Arctic. It seems like they're not very familiar with the warm 
climate of the Rocky Mountains.

Take a second to try out some more invocations and see how the new setup performs. Regardless of your model choice, the following questions should serve as interesting starting points.

In [12]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. Unfortunately, 
without more context, I can't give you a precise location beyond that. However, if you're interested, you can learn
more about them by doing some online research or watching informative documentaries!

In [13]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

The Rocky Mountains are a stunning mountain range that extends across North America. While they're not directly 
close to California, they do reach as far south as New Mexico. If you're traveling from California, you could 
certainly visit some parts of the Rocky Mountains with a bit of journeying. They're truly a sight to behold!

In [14]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

Unfortunately, the context doesn't provide information on Beras' exact location or distance from the Rocky 
Mountains. I do know that Beras mentioned living in the Arctic, which is quite far from the Rockies geographically.
The Rocky Mountains are a vast mountain range located mostly in the western United States and Canada.

<br>

You might notice some decent performance with this always-on retrieval node in the loop since the actual context being fed into the LLM remains relatively small. It's important to experiment with factors like embedding sizes, context limits, and model options to see what kinds of behavior you can expect and which efforts are worth taking to improve performance.

<br>

### **Step 4:** Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

# instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

While I'm sure the Rocky Mountains would provide a magnificent backdrop for enjoying ice cream, it seems like your 
excitement has taken a tasty turn! The Rockies are indeed awe-inspiring, spanning North America and offering 
breathtaking views and adventures. However, let's not forget about that delicious ice cream! While it may not be a 
traditional part of the Rockies experience, there's no reason we can't incorporate it into our dream itinerary. 
After all, who wouldn't enjoy a scoop of their favorite flavor while surrounded by such natural beauty? So, let's 
keep exploring the wonders of the Rocky Mountains, and maybe we'll stumble upon the perfect ice cream spot along 
the way!

While we've been talking about the Rocky Mountains, it seems you've also mentioned that ice cream is such a good 
food. So, I'm guessing your favorite food might just be ice cream! Just remember, though, the Rockies offer plenty 
of natural beauty, hiking trails, and scenic drives to enjoy alongside your favorite treat!

Well, that's a delightful surprise! While ice cream does have its charm, honey sounds equally enjoyable, if not 
more so, considering your preference. It's quite versatile and can be used in many delicious ways. I'm glad we've 
got that cleared up now. Let's continue to explore the wonders of the Rocky Mountains, and perhaps we'll find a way
to incorporate honey into our dream itinerary as well! After all, experiencing new things is what makes travel so 
exciting.

Absolutely, I do! After our little chat, I believe it's safe to say your favorite food is none other than honey. 
It's a delightful choice, indeed! Now, how about we return to our discussion about the Rocky Mountains and see if 
we can't find a way to include your favorite sweet treat in our travel plans? After all, the best part of traveling
is discovering new ways to enjoy our favorite things!

Unlike the more automatic full-text or rule-based approaches to injecting context into the LLM, this approach ensures some amount of consolidation which can keep the context length from getting out of hand. It's still not a full-proof strategy on its own, but it's a stark improvement for unstructured conversations (and doesn't even require a strong instruction-tuned model to perform slot-filling).

----

<br>

## **Part 3 [Exercise]:** RAG For Document Chunk Retrieval

Given our prior exploration of document loading, the idea that data chunks can be embedded and searched through probably isn't surprising. With that said, it is definitely worth going over since applying RAG with documents is a double-edged sword; it may **seem** to work well out of the box but requires some extra care when optimizing it for truly reliable performance. It also provides an excellent opportunity to review some fundamental LCEL skills, so let's see what we can do!

<br>

### **Exercise:**

In the previous example, you may recall that we pulled in some relatively small papers with the help of [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv) using the following syntax:

```python
from langchain.document_loaders import ArxivLoader

docs = [
    ArxivLoader(query="2205.00445").load(),  ## MRKL
    ArxivLoader(query="2210.03629").load(),  ## ReAct
]
```

Given all that you've learned so far, choose a selection of papers that you would like to use and develop a chatbot that can talk about them!

<br>

Though this is a pretty big task, a walkthrough of ***most*** of the process will be provided below. By the end of the walkthrough, many of the necessary puzzle pieces will be provided, and your real task will be to integrate them together for the final `retrieval_chain`. When you're done, get ready to re-integrate the chain (or a flavor of your choice) in the last notebook as part of the evaluation exercise!


<br>

### **Task 1**: Loading And Chunking Your Documents

The following code block gives you some default papers to load in for your RAG chain. Feel free to select more papers as desired, but note that longer documents will take longer to process. A few simplifying assumptions and additional processing steps are included to help you improve your naive RAG performance:

- Documents are cut off prior to the "References" section if one exists. This will keep our system from considering the citations and appendix sections, which tend to be long and distracting.

- A chunk that lists the available documents is inserted to provide a high-level view of all available documents in a single chunk. If your pipeline does not provide metadata on each retrieval, this is a useful component and can even be listed among a list of higher-priority pieces if appropriate.

- Additionally, the metadata entries are also inserted to provide general information. Ideally, there would also be some synthetic chunks that merge the metadata into interesting cross-document chunks.

**NOTE:** ***For the sake of the assessment, please include at least one paper that is less than one month old!***


In [21]:
import json
import requests
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

def load_arxiv_paper(paper_id):
    """Download and load an ArXiv paper"""
    pdf_filename = f"{paper_id}.pdf"
    
    # Download if not already present
    if not os.path.exists(pdf_filename):
        pdf_url = f"https://arxiv.org/pdf/{paper_id}.pdf"
        print(f"Downloading {paper_id}...")
        response = requests.get(pdf_url)
        with open(pdf_filename, 'wb') as f:
            f.write(response.content)
    
    # Load with PyPDFLoader
    loader = PyPDFLoader(pdf_filename)
    return loader.load()

## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [
    load_arxiv_paper("1706.03762"),  ## Attention Is All You Need Paper
    load_arxiv_paper("1810.04805"),  ## BERT Paper
    load_arxiv_paper("2005.11401"),  ## RAG Paper
    load_arxiv_paper("2205.00445"),  ## MRKL Paper
    load_arxiv_paper("2310.06825"),  ## Mistral Paper
    load_arxiv_paper("2306.05685"),  ## LLM-as-a-Judge
    ## Some longer papers
    # load_arxiv_paper("2210.03629"),  ## ReAct Paper
    # load_arxiv_paper("2112.10752"),  ## Latent Stable Diffusion Paper
    # load_arxiv_paper("2103.00020"),  ## CLIP Paper
    ## TODO: Feel free to add more
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title', metadata.get('source', 'Unknown'))
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - 1706.03762.pdf
 - 1810.04805.pdf
 - 2005.11401.pdf
 - 2205.00445.pdf
 - 2310.06825.pdf
 - 2306.05685.pdf 

Document 0
 - # Chunks: 48
 - Metadata: 


{
    'producer': 'pdfTeX-1.40.25',
    'creator': 'LaTeX with hyperref',
    'creationdate': '2024-04-10T21:11:43+00:00',
    'author': '',
    'keywords': '',
    'moddate': '2024-04-10T21:11:43+00:00',
    'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
    'subject': '',
    'title': '',
    'trapped': '/False',
    'source': '1706.03762.pdf',
    'total_pages': 15,
    'page': 0,
    'page_label': '1'
}


Document 1
 - # Chunks: 76
 - Metadata: 


...

{
    'producer': 'pdfTeX-1.40.21',
    'creator': 'LaTeX with hyperref',
    'creationdate': '2022-05-03T01:13:14+00:00',
    'author': '',
    'keywords': '',
    'moddate': '2022-05-03T01:13:14+00:00',
    'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2',
    'subject': '',
    'title': '',
    'trapped': '/False',
    'source': '2205.00445.pdf',
    'total_pages': 19,
    'page': 0,
    'page_label': '1'
}


Document 4
 - # Chunks: 31
 - Metadata: 


{
    'producer': 'pdfTeX-1.40.25',
    'creator': 'LaTeX with hyperref',
    'creationdate': '2023-10-11T00:48:17+00:00',
    'author': '',
    'keywords': '',
    'moddate': '2023-10-11T00:48:17+00:00',
    'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
    'subject': '',
    'title': '',
    'trapped': '/False',
    'source': '2310.06825.pdf',
    'total_pages': 9,
    'page': 0,
    'page_label': '1'
}


Document 5
 - # Chunks: 110
 - Metadata: 


{
    'producer': 'pdfTeX-1.40.25',
    'creator': 'LaTeX with hyperref',
    'creationdate': '2023-12-27T02:05:07+00:00',
    'author': '',
    'keywords': '',
    'moddate': '2023-12-27T02:05:07+00:00',
    'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
    'subject': '',
    'title': '',
    'trapped': '/False',
    'source': '2306.05685.pdf',
    'total_pages': 29,
    'page': 0,
    'page_label': '1'
}

<br>

### **Task 2**: Construct Your Document Vector Stores

Now that we have all of the components, we can go ahead and create indices surrounding them:

In [22]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 866 ms, sys: 48.1 ms, total: 914 ms
Wall time: 32.7 s


<br>

From there, we can combine our indices into a single one using the following utility:

In [23]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 403 chunks


<br>

### **Task 3: [Exercise]** Implement Your RAG Chain

Finally, all the puzzle pieces are in place to implement the RAG pipeline! As a review, we now have:

- A way to construct a from-scratch vector store for conversational memory (and a way to initialize an empty one with `default_FAISS()`)

- A vector store pre-loaded with useful document information from our `ArxivLoader` utility (stored in `docstore`).

With the help of a couple more utilities, you're finally ready to integrate your chain! A few additional convenience utilities are provided (`doc2str` and the now-common `RPrint`) but are optional to use. Additionally, some starter prompts and structures are also defined.

> **Given all of this:** Please implement the `retrieval_chain`.

In [24]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

################################################################################################
## BEGIN TODO: Implement the retrieval chain to make your system work!

retrieval_chain = (
    {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history' : lambda d: None})
    | RunnableAssign({'context' : lambda d: None})
)

retrieval_chain = (
    {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
    | RPrint()
)

## END TODO
################################################################################################

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about RAG!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

{
    'input': 'Tell me about RAG!',
    'history': '',
    'context': '[Quote from Document] is marginalized to get the seq2seq probability p(y|x) via a top-K 
approximation. Concretely, the\ntop K documents are retrieved using the retriever, and the generator produces the 
output sequence\nprobability for each document, which are then marginalized,\npRAG-Sequence(y|x) 
≈\n∑\nz∈top-k(p(·|x))\npη(z|x)pθ(y|x,z) =\n∑\nz∈top-k(p(·|x))\npη(z|x)\nN∏\ni\npθ(yi|x,z,y 1:i−1)\nRAG-Token Model 
In the RAG-Token model we can draw a different latent document for each\ntarget token and marginalize accordingly. 
This allows the generator to choose content from several\ndocuments when producing an answer. Concretely, the top K
documents are retrieved using the\nretriever, and then the generator produces a distribution for the next output 
token for each document,\nbefore marginalizing, and repeating the process with the following output token, 
Formally, we deﬁne:\npRAG-Token(y|x) ≈\nN∏\ni\n∑\nz∈top-k(p(·|x))\npη(z|x)pθ(yi|x,z,y 1:i−1)\n[Quote from Document]
Open-domain question answering (QA) is an important real-world application and common testbed\nfor 
knowledge-intensive tasks [20]. We treat questions and answers as input-output text pairs (x,y)\nand train RAG by 
directly minimizing the negative log-likelihood of answers. We compare RAG to\nthe popular extractive QA paradigm 
[5, 7, 31, 26], where answers are extracted spans from retrieved\ndocuments, relying primarily on non-parametric 
knowledge. We also compare to “Closed-Book\nQA” approaches [52], which, like RAG, generate answers, but which do 
not exploit retrieval, instead\nrelying purely on parametric knowledge. We consider four popular open-domain QA 
datasets: Natural\nQuestions (NQ) [29], TriviaQA (TQA) [24]. WebQuestions (WQ) [3] and CuratedTrec (CT) [2]. As\nCT
and WQ are small, we follow DPR [26] by initializing CT and WQ models with our NQ RAG\nmodel. We use the same 
train/dev/test splits as prior work [ 31, 26] and report Exact Match (EM)\n[Quote from Document] pRAG-Token(y|x) 
≈\nN∏\ni\n∑\nz∈top-k(p(·|x))\npη(z|x)pθ(yi|x,z,y 1:i−1)\nFinally, we note that RAG can be used for sequence 
classiﬁcation tasks by considering the target class\nas a target sequence of length one, in which case RAG-Sequence
and RAG-Token are equivalent.\n2.2 Retriever: DPR\nThe retrieval component pη(z|x) is based on DPR [26]. DPR 
follows a bi-encoder architecture:\npη(z|x) ∝exp\n(\nd(z)⊤q(x)\n)\nd(z) =BERTd(z), q(x) =BERTq(x)\nwhere d(z) is a 
dense representation of a document produced by a BERTBASE document encoder [8],\nand q(x) a query representation 
produced by a query encoder, also based on BERTBASE. Calculating\ntop-k(pη(·|x)), the list of kdocuments zwith 
highest prior probability pη(z|x), is a Maximum Inner\nProduct Search (MIPS) problem, which can be approximately 
solved in sub-linear time [23]. We use\na pre-trained bi-encoder from DPR to initialize our retriever and to build 
the document index. This\n[Quote from Document] tasks, RAG sets a new state of the art (only on the T5-comparable 
split for TQA). RAG combines\nthe generation ﬂexibility of the “closed-book” (parametric only) approaches and the 
performance of\n"open-book" retrieval-based approaches. Unlike REALM and T5+SSM, RAG enjoys strong results\nwithout
expensive, specialized “salient span masking” pre-training [20]. It is worth noting that RAG’s\nretriever is 
initialized using DPR’s retriever, which uses retrieval supervision on Natural Questions\nand TriviaQA. RAG 
compares favourably to the DPR QA system, which uses a BERT-based “cross-\nencoder” to re-rank documents, along 
with an extractive reader. RAG demonstrates that neither a\nre-ranker nor extractive reader is necessary for 
state-of-the-art performance.\nThere are several advantages to generating answers even when it is possible to 
extract them. Docu-\nments with clues about the answer but do not contain the answer verbatim can still contribute 
toward

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Tell me about RAG!\n\n From this, we have retrieved the following potentially-useful info:  
Conversation History Retrieval:\n\n\n Document Retrieval:\n[Quote from Document] is marginalized to get the seq2seq
probability p(y|x) via a top-K approximation. Concretely, the\ntop K documents are retrieved using the retriever, 
and the generator produces the output sequence\nprobability for each document, which are then 
marginalized,\npRAG-Sequence(y|x) ≈\n∑\nz∈top-k(p(·|x))\npη(z|x)pθ(y|x,z) 
=\n∑\nz∈top-k(p(·|x))\npη(z|x)\nN∏\ni\npθ(yi|x,z,y 1:i−1)\nRAG-Token Model In the RAG-Token model we can draw a 
different latent document for each\ntarget token and marginalize accordingly. This allows the generator to choose 
content from several\ndocuments when producing an answer. Concretely, the top K documents are retrieved using 
the\nretriever, and then the generator produces a distribution for the next output token for each document,\nbefore
marginalizing, and repeating the process with the following output token, Formally, we deﬁne:\npRAG-Token(y|x) 
≈\nN∏\ni\n∑\nz∈top-k(p(·|x))\npη(z|x)pθ(yi|x,z,y 1:i−1)\n[Quote from Document] Open-domain question answering (QA) 
is an important real-world application and common testbed\nfor knowledge-intensive tasks [20]. We treat questions 
and answers as input-output text pairs (x,y)\nand train RAG by directly minimizing the negative log-likelihood of 
answers. We compare RAG to\nthe popular extractive QA paradigm [5, 7, 31, 26], where answers are extracted spans 
from retrieved\ndocuments, relying primarily on non-parametric knowledge. We also compare to “Closed-Book\nQA” 
approaches [52], which, like RAG, generate answers, but which do not exploit retrieval, instead\nrelying purely on 
parametric knowledge. We consider four popular open-domain QA datasets: Natural\nQuestions (NQ) [29], TriviaQA 
(TQA) [24]. WebQuestions (WQ) [3] and CuratedTrec (CT) [2]. As\nCT and WQ are small, we follow DPR [26] by 
initializing CT and WQ models with our NQ RAG\nmodel. We use the same train/dev/test splits as prior work [ 31, 26]
and report Exact Match (EM)\n[Quote from Document] pRAG-Token(y|x) ≈\nN∏\ni\n∑\nz∈top-k(p(·|x))\npη(z|x)pθ(yi|x,z,y
1:i−1)\nFinally, we note that RAG can be used for sequence classiﬁcation tasks by considering the target class\nas 
a target sequence of length one, in which case RAG-Sequence and RAG-Token are equivalent.\n2.2 Retriever: DPR\nThe 
retrieval component pη(z|x) is based on DPR [26]. DPR follows a bi-encoder architecture:\npη(z|x) 
∝exp\n(\nd(z)⊤q(x)\n)\nd(z) =BERTd(z), q(x) =BERTq(x)\nwhere d(z) is a dense representation of a document produced 
by a BERTBASE document encoder [8],\nand q(x) a query representation produced by a query encoder, also based on 
BERTBASE. Calculating\ntop-k(pη(·|x)), the list of kdocuments zwith highest prior probability pη(z|x), is a Maximum
Inner\nProduct Search (MIPS) problem, which can be approximately solved in sub-linear time [23]. We use\na 
pre-trained bi-encoder from DPR to initialize our retriever and to build the document index. This\n[Quote from 
Document] tasks, RAG sets a new state of the art (only on the T5-comparable split for TQA). RAG combines\nthe 
generation ﬂexibility of the “closed-book” (parametric only) approaches and the performance of\n"open-book" 
retrieval-based approaches. Unlike REALM and T5+SSM, RAG enjoys strong results\nwithout expensive, specialized 
“salient span masking” pre-training [20]. It is worth noting that RAG’s\nretriever is initialized using DPR’s 
retriever, which uses retrieval supervision on Natural Questions\nand TriviaQA. RAG compares favourably to the DPR 
QA system, which uses a BERT-based “cross-\nencoder” to re-rank documents, along with an extractive reader. RAG 
demonstrates that neither a\nre-ranker nor extra

RAG (Retrieval-Augmented Generation) is a model used for open-domain question answering and other knowledge-intensive tasks. It combines the flexibility of "closed-book" approaches, which generate answers based on parametric knowledge, with the performance of "open-book" retrieval-based approaches, which extract answers from retrieved documents. RAG sets a new state of the art in open-domain question answering on several popular datasets, including Natural Questions and TriviaQA.

The model consists of two main components: a retriever and a generator. The retriever, which is based on DPR (Dense Passage Retrieval), is responsible for retrieving the most relevant documents for a given input sequence. It does this by generating a dense representation of the input sequence and comparing it to dense representations of all available documents. The top K documents are then passed to the generator.

The generator, which is based on a sequence-to-sequence model, produces the final answer by gen

### **Task 4:** Interact With Your Gradio Chatbot

In [25]:
# chatbot = gr.Chatbot(value = [[None, initial_msg]])
# demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

# try:
#     demo.launch(debug=True, share=True, show_api=False)
#     demo.close()
# except Exception as e:
#     demo.close()
#     print(e)
#     raise e

<br>

----

<br>

## **Part 4:** Saving Your Index For Evaluation

After you've implemented your RAG chain, please save your accumulated vector store as shown [in the official documentation](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading). You'll have a chance to use it again for your final assessment!

In [26]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


If everything was properly saved, the following line can be invoked to pull the index from the compressed `tgz` file (assuming the pip requirements are installed). After you have confirmed that the cell can pull in your index, download `docstore_index.tgz` for use in the last notebook!

In [27]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
p(z|x). Table 6 shows the results. For FEVER, BM25 performs best, perhaps since FEVER claims are
heavily entity-centric and thus well-suited for word overlap-based retrieval. Differentiable retrieval
improves results on all other tasks, especially for Open-Domain QA, where it is crucial.
Index hot-swapping An advantage of non-parametric memory models like RAG is that knowledge
can be easily updated at test time. Parametric-only models like T5 or BART need further training to
update their behavior as the world changes. To demonstrate, we build an index using the DrQA [5]
Wikipedia dump from December 2016 and compare outputs from RAG using this index to the newer
index from our main results (December 2018). We prepare a list of 82 world leaders who had changed
7


-----

<br>

## **Part 5:** Wrap-Up

Congratulations! Assuming your RAG chain is all good, you're now ready to move on to the **RAG Evaluation [Assessment]** section!

### <font color="#76b900">**Great Job!**</font>

### **Next Steps:**
1. **[Optional]** Revisit the **"Questions To Think About" Section** at the top of the notebook and think about some possible answers.

---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>